In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import numpy as np
import torch
import gpytorch
import matplotlib.pyplot as plt
import tqdm
import utils as utils
import scienceplots
plt.style.use('science')

base_dir = os.path.join(os.getcwd(), '..')
sys.path.append(base_dir)

import fit_FaIRGP as globalfairgp
import fit_spatial_FaIRGP as spatialfairgp

<IPython.core.display.Javascript object>

## Global

In [7]:
train_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['historical', 'ssp126', 'ssp370', 'ssp585']}}
train_data = globalfairgp.make_data(train_cfg)

test_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['ssp245']}}
test_data = globalfairgp.make_data(test_cfg)
test_scenarios = test_data.scenarios

fairgpmodel = globalfairgp.make_model(train_cfg, train_data)
state_dict = torch.load('../data/experiments/outputs/ssps/GFaIR/fold_ssp245/state_dict.pt')
fairgpmodel.load_state_dict(state_dict)
fairgpmodel = fairgpmodel.eval()

In [8]:
%%timeit
with torch.no_grad():
    fairgp_test_posterior = fairgpmodel(test_scenarios)
    noisy_fairgp_test_posterior = fairgpmodel.likelihood(fairgp_test_posterior)

/home/bouabid/.pyenv/versions/sandbox/lib/python3.8/site-packages/gpytorch/lazy/lazy_tensor.py:25: UserWarning: gpytorch.lazy.delazify is deprecated in favor of linear_operator.to_dense
  warnings.warn("gpytorch.lazy.delazify is deprecated in favor of linear_operator.to_dense")
/home/bouabid/.pyenv/versions/sandbox/lib/python3.8/site-packages/gpytorch/lazy/non_lazy_tensor.py:13: UserWarning: gpytorch.lazy.lazify is deprecated in favor of linear_operator.to_linear_operator
  warnings.warn("gpytorch.lazy.lazify is deprecated in favor of linear_operator.to_linear_operator")


923 ms ± 17.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Spatial

In [10]:
train_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['historical', 'ssp126', 'ssp370', 'ssp585']},}
train_data = spatialfairgp.make_data(train_cfg)

test_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['ssp245']}}
test_data = spatialfairgp.make_data(test_cfg)
test_scenarios = test_data.scenarios

model = spatialfairgp.make_model(train_cfg, train_data)
state_dict = torch.load('../data/models/leave-one-out-ssp/ssp245/SpatialFaIRGP/state_dict.pt')
model.load_state_dict(state_dict)
model = model.eval()

In [13]:
%%timeit
with torch.no_grad():
    posterior = model.posterior(test_scenarios, diag=False)
    posterior = model.likelihood(posterior)

682 ms ± 32.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
